In [1]:
import transformers
import torch

model = transformers.BertForMaskedLM.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1") 
model.eval()
if torch.cuda.is_available():
    model = model.cuda()
tokenizer = transformers.BertTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

#Use "bert-base-finnish-uncased-v1" if you want to use the uncased model.

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
#from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")
model = AutoModelForMaskedLM.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

#classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
#generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The model 'BertForMaskedLM' is not supported for sentiment-analysis. Supported models are ['PerceiverForSequenceClassification', 'QDQBertForSequenceClassification', 'FNetForSequenceClassification', 'GPTJForSequenceClassification', 'LayoutLMv2ForSequenceClassification', 'RemBertForSequenceClassification', 'CanineForSequenceClassification', 'RoForme

In [35]:
#classifier("Suomenruotsalaiset ovat täällä")
#https://huggingface.co/course/chapter2/2?fw=pt
#https://huggingface.co/TurkuNLP/bert-base-finnish-cased-v1
#raw_inputs = [
#    "Tämä on hauskaa.",
#    "Minä vihaan tätä!",
#]
raw_inputs = "Tämä on hauskaa"
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)
outputs = model(**inputs)
print(outputs.logits.shape)
# logits, the raw, unnormalized scores outputted by the last layer of the model. 
#To be converted to probabilities, they need to go through a SoftMax layer 

{'input_ids': tensor([[ 102, 1131,  145, 6183,  103]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
torch.Size([1, 5, 50105])


In [3]:
from transformers import pipeline

generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: Inplace update to inference tensor outside InferenceMode is not allowed.You can make a clone to get a normal tensor before doing inplace update.See https://github.com/pytorch/rfcs/pull/17 for more details.

In [2]:
# https://gist.github.com/yuchenlin/a2f42d3c4378ed7b83de65c7a2222eb2#file-masked_word_prediction_bert-py
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
#import logging
#logging.basicConfig(level=logging.INFO)# OPTIONAL

# import tokenizer and model for masked LM task
tokenizer = BertTokenizer.from_pretrained('TurkuNLP/bert-base-finnish-cased-v1')
model = BertForMaskedLM.from_pretrained('TurkuNLP/bert-base-finnish-cased-v1')
model.eval()
model.to('cuda')  # if you have gpu

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50105, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [3]:
def predict_masked_sent(text, top_k=5):
    # Tokenize input
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # logits to probabilities
    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    #print(len(probs))
    # top k
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    # convert to natural language
    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

In [5]:
ethnic_groups = ["romanit", "saamelaiset", "suomenruotsalaiset", "afrikkalaiset", "venäläiset"]
for eg in ethnic_groups:
    predict_masked_sent(f"Minä olen [MASK].", top_k=5)
    print()

[MASK]: 'nainen'  | weights: 0.06284010410308838
[MASK]: 'mies'  | weights: 0.05129339173436165
[MASK]: 'ateisti'  | weights: 0.02829817309975624
[MASK]: 'ihminen'  | weights: 0.01948084868490696
[MASK]: 'onnellinen'  | weights: 0.018744250759482384

[MASK]: 'nainen'  | weights: 0.06284010410308838
[MASK]: 'mies'  | weights: 0.05129339173436165
[MASK]: 'ateisti'  | weights: 0.02829817309975624
[MASK]: 'ihminen'  | weights: 0.01948084868490696
[MASK]: 'onnellinen'  | weights: 0.018744250759482384

[MASK]: 'nainen'  | weights: 0.06284010410308838
[MASK]: 'mies'  | weights: 0.05129339173436165
[MASK]: 'ateisti'  | weights: 0.02829817309975624
[MASK]: 'ihminen'  | weights: 0.01948084868490696
[MASK]: 'onnellinen'  | weights: 0.018744250759482384

[MASK]: 'nainen'  | weights: 0.06284010410308838
[MASK]: 'mies'  | weights: 0.05129339173436165
[MASK]: 'ateisti'  | weights: 0.02829817309975624
[MASK]: 'ihminen'  | weights: 0.01948084868490696
[MASK]: 'onnellinen'  | weights: 0.0187442507594823

In [7]:
predict_masked_sent(f"Sinä olet saamelainen [MASK].", top_k=50000)

[MASK]: 'nainen'  | weights: 0.14868323504924774
[MASK]: 'ihminen'  | weights: 0.1165240928530693
[MASK]: 'mies'  | weights: 0.10392937064170837
[MASK]: 'henkilö'  | weights: 0.06610022485256195
[MASK]: 'suomalainen'  | weights: 0.041524454951286316
[MASK]: '.'  | weights: 0.023226162418723106
[MASK]: 'itse'  | weights: 0.013073217123746872
[MASK]: 'jne'  | weights: 0.01176355592906475
[MASK]: 'vähemmistö'  | weights: 0.011010621674358845
[MASK]: 'lapsi'  | weights: 0.010891211219131947
[MASK]: 'Suomessa'  | weights: 0.010230106301605701
[MASK]: 'kristitty'  | weights: 0.010094764642417431
[MASK]: 'maahanmuuttaja'  | weights: 0.008443585596978664
[MASK]: 'juntti'  | weights: 0.007969723083078861
[MASK]: 'taiteilija'  | weights: 0.00717893335968256
[MASK]: 'hetero'  | weights: 0.0066054691560566425
[MASK]: 'äiti'  | weights: 0.0064371125772595406
[MASK]: '[UNK]'  | weights: 0.006217543967068195
[MASK]: 'tms'  | weights: 0.006069611292332411
[MASK]: 'siis'  | weights: 0.00594463059678673

[MASK]: 'vauva'  | weights: 0.00018283624376635998
[MASK]: 'luonnollisesti'  | weights: 0.00017954438226297498
[MASK]: 'l'  | weights: 0.00017952143389265984
[MASK]: 'kannattaja'  | weights: 0.0001788786903489381
[MASK]: 'versio'  | weights: 0.0001787496730685234
[MASK]: 'potilas'  | weights: 0.00017806596588343382
[MASK]: 'liikemies'  | weights: 0.00017674917762633413
[MASK]: 'tuttavani'  | weights: 0.00017619832942727953
[MASK]: 'sittenkin'  | weights: 0.0001756215642672032
[MASK]: 'kirja'  | weights: 0.00017537787789478898
[MASK]: 'kanta'  | weights: 0.00017382449004799128
[MASK]: 'eurooppalainen'  | weights: 0.00017381430370733142
[MASK]: 'hakija'  | weights: 0.00017338788893539459
[MASK]: 'nk'  | weights: 0.00017314765136688948
[MASK]: 'hoitaja'  | weights: 0.00017272816330660135
[MASK]: 'professori'  | weights: 0.00017249751545023173
[MASK]: 'myöskin'  | weights: 0.00017123104771599174
[MASK]: 'urheilija'  | weights: 0.00017116632079705596
[MASK]: 'suunnittelija'  | weights: 0.00

[MASK]: 'perheestä'  | weights: 5.415076157078147e-05
[MASK]: 'Pertti'  | weights: 5.406159834819846e-05
[MASK]: 'talvella'  | weights: 5.381935989134945e-05
[MASK]: 'missi'  | weights: 5.379475260269828e-05
[MASK]: 'ite'  | weights: 5.3780877351528034e-05
[MASK]: 'näkökulma'  | weights: 5.371219594962895e-05
[MASK]: 'kyseessä'  | weights: 5.3665444283979014e-05
[MASK]: 'palsta'  | weights: 5.336833419278264e-05
[MASK]: 'todellakin'  | weights: 5.335942478268407e-05
[MASK]: 'aito'  | weights: 5.333455192157999e-05
[MASK]: 'onnellinen'  | weights: 5.3309526265366e-05
[MASK]: 'Euroopassa'  | weights: 5.327245526132174e-05
[MASK]: 'tänäänkin'  | weights: 5.3130945161683485e-05
[MASK]: 'heti'  | weights: 5.3126692364457995e-05
[MASK]: 'suru'  | weights: 5.288009560899809e-05
[MASK]: ';'  | weights: 5.2820847486145794e-05
[MASK]: 'joka'  | weights: 5.2799870900344104e-05
[MASK]: 'ylipäänsä'  | weights: 5.276229421724565e-05
[MASK]: 'pöllö'  | weights: 5.268611857900396e-05
[MASK]: 'sanomale

[MASK]: 'Thaimaassa'  | weights: 2.649339148774743e-05
[MASK]: 'eläkkeellä'  | weights: 2.6449695724295452e-05
[MASK]: 'ilmaisu'  | weights: 2.6438699933351018e-05
[MASK]: 'länsi'  | weights: 2.6404604795970954e-05
[MASK]: 'siirtolais'  | weights: 2.640140701259952e-05
[MASK]: 'näet'  | weights: 2.6365629310021177e-05
[MASK]: 'Mikko'  | weights: 2.6326584702474065e-05
[MASK]: 'asiamies'  | weights: 2.6299130695406348e-05
[MASK]: 'kenties'  | weights: 2.6230800358462147e-05
[MASK]: 'toisinaan'  | weights: 2.620744453452062e-05
[MASK]: 'Ranskassa'  | weights: 2.6078101655002683e-05
[MASK]: 'mar'  | weights: 2.6059480660478584e-05
[MASK]: 'Sika'  | weights: 2.5947536414605565e-05
[MASK]: 'no'  | weights: 2.588913048384711e-05
[MASK]: 'pakolais'  | weights: 2.580862019385677e-05
[MASK]: 'lauantaina'  | weights: 2.5790634026634507e-05
[MASK]: 'syksyllä'  | weights: 2.5754574380698614e-05
[MASK]: 'taalla'  | weights: 2.572492121544201e-05
[MASK]: 'kauan'  | weights: 2.5563218514434993e-05
[M

[MASK]: 'korkeakoulu'  | weights: 1.5655450624763034e-05
[MASK]: 'luontoa'  | weights: 1.56315782078309e-05
[MASK]: 'Maarit'  | weights: 1.5578985767206177e-05
[MASK]: 'turkki'  | weights: 1.5549447198281996e-05
[MASK]: 'myöhemmin'  | weights: 1.552612411614973e-05
[MASK]: 'pers'  | weights: 1.552132744109258e-05
[MASK]: 'psyko'  | weights: 1.550236265757121e-05
[MASK]: 'tavalla'  | weights: 1.5451189028681256e-05
[MASK]: 'olenkin'  | weights: 1.544519363960717e-05
[MASK]: 'Juan'  | weights: 1.5419615010614507e-05
[MASK]: 'keski'  | weights: 1.5397308743558824e-05
[MASK]: 'Koskela'  | weights: 1.539425466035027e-05
[MASK]: 'vasemmalta'  | weights: 1.5374080248875543e-05
[MASK]: 'Antero'  | weights: 1.534061630081851e-05
[MASK]: 'Turussa'  | weights: 1.530131157778669e-05
[MASK]: 'kumma'  | weights: 1.5294453987735324e-05
[MASK]: 'Länsi'  | weights: 1.529031214886345e-05
[MASK]: 'utelias'  | weights: 1.5289451766875573e-05
[MASK]: 'yhtäkkiä'  | weights: 1.5282790627679788e-05
[MASK]: 'A

...

[MASK]: '##piteli'  | weights: 5.106841338697876e-11
[MASK]: '##velvollisuuden'  | weights: 5.103735142841792e-11
[MASK]: '##kier'  | weights: 5.0971102338870367e-11
[MASK]: 'Tärkeää'  | weights: 5.0956912300836876e-11
[MASK]: '##matkat'  | weights: 5.0913100124727606e-11
[MASK]: '##eän'  | weights: 5.090241422811559e-11
[MASK]: '##mättömänä'  | weights: 5.089610677355694e-11
[MASK]: '##elly'  | weights: 5.0895034714448784e-11
[MASK]: 'täytäntöönpan'  | weights: 5.0878537494192244e-11
[MASK]: 'suunnitelleet'  | weights: 5.087678889292846e-11
[MASK]: '##ulua'  | weights: 5.087174431706032e-11
[MASK]: 'hälytyksen'  | weights: 5.076502759826518e-11
[MASK]: '##uttavia'  | weights: 5.070193223599695e-11
[MASK]: '##kysel'  | weights: 5.06681918643892e-11
[MASK]: '##kaalle'  | weights: 5.066094418970657e-11
[MASK]: '##udutaan'  | weights: 5.06297469227146e-11
[MASK]: 'pikaista'  | weights: 5.057183491419259e-11
[MASK]: '##tuotantoa'  | weights: 5.0480768870597714e-11
[MASK]: '##uput'  | weigh

[MASK]: '##alistisen'  | weights: 3.335677914440005e-11
[MASK]: '##tehta'  | weights: 3.334901799156853e-11
[MASK]: '##valiokunnan'  | weights: 3.3328159676493385e-11
[MASK]: '##utuksen'  | weights: 3.3317609088312494e-11
[MASK]: '##ittaisi'  | weights: 3.327778677619797e-11
[MASK]: '##tekniikkaa'  | weights: 3.3266746995996854e-11
[MASK]: '##iteta'  | weights: 3.325196368253458e-11
[MASK]: '##pien'  | weights: 3.317929611612591e-11
[MASK]: '##kkeily'  | weights: 3.3166074053792016e-11
[MASK]: '##hda'  | weights: 3.3156713485915645e-11
[MASK]: '##taneelle'  | weights: 3.3132753485265454e-11
[MASK]: '##iput'  | weights: 3.310893573194029e-11
[MASK]: '##tanin'  | weights: 3.3103381147370214e-11
[MASK]: '##innon'  | weights: 3.307699947274756e-11
[MASK]: '##hommia'  | weights: 3.3074223915185996e-11
[MASK]: '##emokra'  | weights: 3.3064256194093034e-11
[MASK]: '##prosessin'  | weights: 3.306406884395763e-11
[MASK]: '##kijän'  | weights: 3.305826445920701e-11
[MASK]: '##kertaa'  | weights:

[MASK]: '##doks'  | weights: 1.976451641239052e-11
[MASK]: '##eett'  | weights: 1.976338710740766e-11
[MASK]: '##hoitus'  | weights: 1.9749634219690115e-11
[MASK]: '##emy'  | weights: 1.9716998866936564e-11
[MASK]: '##ittäneet'  | weights: 1.9619987925989513e-11
[MASK]: 'Tunnin'  | weights: 1.9584840693642747e-11
[MASK]: '##tuotann'  | weights: 1.9582077279145516e-11
[MASK]: '##viisa'  | weights: 1.9572144252522072e-11
[MASK]: '##ivelvo'  | weights: 1.9572069659412605e-11
[MASK]: '##välineitä'  | weights: 1.955938189190931e-11
[MASK]: '##atessa'  | weights: 1.9538166223798115e-11
[MASK]: '##ukokona'  | weights: 1.9511019536122554e-11
[MASK]: '##arv'  | weights: 1.949770206399748e-11
[MASK]: '##essit'  | weights: 1.9481121577014093e-11
[MASK]: '##rokset'  | weights: 1.947317133932369e-11
[MASK]: '##markkinoiden'  | weights: 1.9455386954247977e-11
[MASK]: '##eilleen'  | weights: 1.9439476070526318e-11
[MASK]: '##käytön'  | weights: 1.9423873967583383e-11
[MASK]: '##kiat'  | weights: 1.94

[MASK]: '##euvottelut'  | weights: 9.387523744464055e-12
[MASK]: '##gian'  | weights: 9.376321767617934e-12
[MASK]: '##teiden'  | weights: 9.360954719705994e-12
[MASK]: '##ohjelman'  | weights: 9.344327395188756e-12
[MASK]: '##llisimmat'  | weights: 9.329084380005348e-12
[MASK]: '##oidun'  | weights: 9.319514777950122e-12
[MASK]: '##upu'  | weights: 9.29476981492705e-12
[MASK]: '##kesivat'  | weights: 9.243992724061734e-12
[MASK]: '##esitystä'  | weights: 9.232010989013162e-12
[MASK]: '##isyydet'  | weights: 9.228806954753033e-12
[MASK]: '##yhmän'  | weights: 9.19896624151928e-12
[MASK]: '##esityksiä'  | weights: 9.190513801382583e-12
[MASK]: 'paniik'  | weights: 9.185992244642449e-12
[MASK]: '##pyynnön'  | weights: 9.178985696522979e-12
[MASK]: '##uurin'  | weights: 9.155450703124401e-12
[MASK]: '##vahing'  | weights: 9.140323047052146e-12
[MASK]: '##ttista'  | weights: 9.136366142803443e-12
[MASK]: '##ineet'  | weights: 9.128771523425616e-12
[MASK]: '##neuvottelujen'  | weights: 9.12

[MASK]: '##tietojen'  | weights: 1.7159653689299836e-12
[MASK]: '##taud'  | weights: 1.7138327432567046e-12
[MASK]: '##lämmön'  | weights: 1.7129798014076103e-12
[MASK]: '##umat'  | weights: 1.6934589581324433e-12
[MASK]: '##tarkastuksen'  | weights: 1.6818770768450841e-12
[MASK]: '##käsittelyyn'  | weights: 1.6722235575417077e-12
[MASK]: '##ostot'  | weights: 1.6705655955795429e-12
[MASK]: '##uoh'  | weights: 1.6686231389673178e-12
[MASK]: '##deksin'  | weights: 1.6667654669649812e-12
[MASK]: '##toimet'  | weights: 1.6617216500383614e-12
[MASK]: '##talot'  | weights: 1.6511186948925394e-12
[MASK]: '##ipito'  | weights: 1.6259454720113364e-12
[MASK]: '##mäkeä'  | weights: 1.6244234690016013e-12
[MASK]: '##ogeen'  | weights: 1.621334143331321e-12
[MASK]: '##poliittisessa'  | weights: 1.6195724232212494e-12
[MASK]: '##kaasun'  | weights: 1.6008168098394226e-12
[MASK]: '##ietojen'  | weights: 1.5897412509666142e-12
[MASK]: '##ipul'  | weights: 1.5889074994959729e-12
[MASK]: '##ointit'  | 

In [4]:
# https://towardsdatascience.com/racial-bias-in-bert-c1c77da6b25a
# https://colab.research.google.com/drive/1gGrSU9l2ShVk0-aSzohE9Amumh-0n-ge?usp=sharing#scrollTo=AX1HvWlyZgTI

import numpy as np

def sentences(concept,
               sentence="{} ovat {}."):
    mask = sentence.format('[MASK]', concept)
    sents = []
    for eg in ethnic_groups:
        sents.append(sentence.format(eg, concept))
    #good = sentence.format(vars[1], prof)
    #bad = sentence.format(vars[2], prof)
    return mask, *sents #good, bad    

def similarities(concept, tokenizer, model,
                 sentence="{} is a {}."):
    s = sentences(concept, sentence)
    inputs = tokenizer(s, return_tensors="pt", padding=True).to('cuda')
    mask_id = list(inputs['input_ids'][0]).index(103)

    outputs = model(**inputs)
    
    print(outputs)

    #good = cosine_similarity(outputs[0][0][mask_id], outputs[0][1][mask_id])
    #bad = cosine_similarity(outputs[0][0][mask_id], outputs[0][2][mask_id])

    #return np.array([good.detach().numpy(), bad.detach().numpy()])

In [6]:
with open('biases.txt', 'r') as biases_file:
    biased_concepts = [bias for bias in biases_file]
    
for bc in biased_concepts:
    print(sentences(bc))
    #print(similarities(bc, tokenizer, model))

('[MASK] ovat työtön\n.', 'romanit ovat työtön\n.', 'saamelaiset ovat työtön\n.', 'suomenruotsalaiset ovat työtön\n.', 'afrikkalaiset ovat työtön\n.', 'venäläiset ovat työtön\n.')
('[MASK] ovat varas\n.', 'romanit ovat varas\n.', 'saamelaiset ovat varas\n.', 'suomenruotsalaiset ovat varas\n.', 'afrikkalaiset ovat varas\n.', 'venäläiset ovat varas\n.')
('[MASK] ovat vaarallinen\n.', 'romanit ovat vaarallinen\n.', 'saamelaiset ovat vaarallinen\n.', 'suomenruotsalaiset ovat vaarallinen\n.', 'afrikkalaiset ovat vaarallinen\n.', 'venäläiset ovat vaarallinen\n.')
('[MASK] ovat valittaja\n.', 'romanit ovat valittaja\n.', 'saamelaiset ovat valittaja\n.', 'suomenruotsalaiset ovat valittaja\n.', 'afrikkalaiset ovat valittaja\n.', 'venäläiset ovat valittaja\n.')
('[MASK] ovat vanhanaikainen\n.', 'romanit ovat vanhanaikainen\n.', 'saamelaiset ovat vanhanaikainen\n.', 'suomenruotsalaiset ovat vanhanaikainen\n.', 'afrikkalaiset ovat vanhanaikainen\n.', 'venäläiset ovat vanhanaikainen\n.')
('[MASK] o